# Objetivo: formar um portfólio ótimo com base em uma carteira customizada de ativos listados na Bovespa

# Importando pacotes

In [4]:
!pip install yfinance

In [5]:
import pandas as pd
import numpy as np
import requests
import yfinance as yf
import datetime as dt
from pandas.tseries.offsets import MonthEnd

# Carregando dados

In [6]:
tickers = pd.read_csv('tickers.csv', sep=";")
tickers = pd.DataFrame(tickers)
tickers = tickers['0'].to_list()

In [7]:
df = yf.download(tickers, start='2019-1-2',end='2021-1-6')

[*********************100%***********************]  75 of 75 completed


In [8]:
df = df['Adj Close']
df

,ABEV3.SA,AZUL4.SA,B3SA3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BEEF3.SA,BPAC11.SA,BRAP4.SA,BRDT3.SA,BRFS3.SA,BRKM5.SA,BRML3.SA,BTOW3.SA,CCRO3.SA,CIEL3.SA,CMIG4.SA,COGN3.SA,CPFE3.SA,CRFB3.SA,CSAN3.SA,CSNA3.SA,CVCB3.SA,CYRE3.SA,ECOR3.SA,EGIE3.SA,ELET3.SA,ELET6.SA,EMBR3.SA,ENBR3.SA,ENGI11.SA,EQTL3.SA,FLRY3.SA,GGBR4.SA,GNDI3.SA,GOAU4.SA,GOLL4.SA,HAPV3.SA,HGTX3.SA,HYPE3.SA,IGTA3.SA,IRBR3.SA,ITSA4.SA,ITUB4.SA,JBSS3.SA,KLBN11.SA,LAME4.SA,LREN3.SA,MGLU3.SA,MRFG3.SA,MRVE3.SA,MULT3.SA,NTCO3.SA,PCAR3.SA,PETR3.SA,PETR4.SA,QUAL3.SA,RADL3.SA,RAIL3.SA,RENT3.SA,SANB11.SA,SBSP3.SA,SULA11.SA,SUZB3.SA,TAEE11.SA,TIMP3.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,VIVT4.SA,VVAR3.SA,WEGE3.SA,YDUQ3.SA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-02,15.736688,36.330002,26.106749,43.852047,27.576366,30.612583,23.910286,4.849414,23.484318,28.664137,22.239382,22.379999,43.746208,12.521657,42.501801,10.978712,8.855656,12.414179,9.141588,25.250971,18.473175,34.207909,8.202543,60.521698,13.678994,9.72,30.526434,21.194765,27.712151,21.540001,13.523678,38.000000,14.426986,18.553518,14.691135,29.321878,6.878463,25.450001,5.816300,28.805285,29.003435,41.089645,26.375177,11.378636,34.309326,11.766559,15.480146,19.130684,38.375622,5.543480,5.65,11.134134,24.063623,NaN,83.534805,25.234716,23.284786,12.644371,11.186800,16.950001,28.884201,41.300167,32.435802,26.396025,37.770512,20.981518,11.415798,28.291004,26.168896,9.508233,47.774075,40.703033,4.380000,17.561279,23.945028
2019-01-03,15.912081,36.369999,27.243458,44.032516,28.151554,30.921385,24.172014,4.963963,24.455757,27.566988,22.752602,21.490000,44.008808,12.605382,42.936699,11.335520,8.644151,12.153697,9.250298,24.945587,19.262793,34.725342,8.300510,60.135391,13.903096,9.92,31.527895,27.049669,29.378660,21.900000,14.086369,38.200001,14.829500,18.949142,14.594483,29.262362,6.907283,25.629999,5.683517,29.196222,29.422426,41.467331,26.064850,11.477582,34.874977,11.952243,15.878778,19.072474,37.890965,5.484165,5.69,11.125324,24.237436,NaN,83.534805,25.755508,23.855778,12.718532,11.161151,17.440001,28.407026,42.663597,34.936676,25.691505,36.155972,21.042187,11.346310,28.942684,25.682505,9.303650,45.819725,41.065376,4.310000,17.961729,23.925756
2019-01-04,16.126450,36.250000,26.257055,44.032516,27.534498,30.678808,23.935616,5.154885,25.323452,29.429375,22.838137,21.450001,43.519821,12.363507,41.513401,11.207435,9.333842,12.032130,9.260180,24.997938,18.716883,34.524113,8.683474,59.382584,13.782425,9.96,32.204773,26.639582,29.172493,20.799999,14.191278,38.500000,15.022091,19.319788,14.981093,29.272280,7.080205,25.000000,5.704089,28.290501,29.282766,40.431129,25.773520,11.441605,34.290787,12.264977,16.135040,18.829987,36.437019,5.317255,5.68,10.966767,23.744963,NaN,83.534805,26.049042,23.923521,12.496049,11.320961,17.200001,28.116905,42.618450,35.729404,24.536093,36.762650,21.874165,11.403420,29.742573,25.509134,9.849203,48.802681,40.979103,4.270000,18.166838,24.291916
2019-01-07,16.058243,35.299999,26.116142,43.743771,27.765879,31.042761,23.581011,5.126246,25.842182,29.659866,22.966440,21.590000,43.211941,12.289084,39.674900,10.887221,9.251082,11.676203,9.022994,24.675104,18.375690,34.016266,8.701287,60.125484,13.704852,9.84,32.005310,25.304564,27.918318,21.230000,13.781179,38.000000,15.118386,19.597767,15.019753,28.865583,7.128239,23.900000,5.790117,27.348448,28.817217,39.801662,26.033220,11.432607,34.355686,11.834969,16.135040,19.458168,35.599922,5.399814,5.58,11.063664,23.406992,NaN,83.534805,26.891781,24.300957,12.125246,11.068420,16.900000,27.592960,43.331768,34.917797,24.930624,35.079609,21.510176,11.774652,29.100685,24.728981,9.868688,48.540855,41.324192,4.240000,17.883595,23.810125
2019-01-08,15.736688,34.720001,26.191290,43.247505,28.074360,31.232302,23.234854,5.068967,25.360952,29.447817,22.196617,22.930000,43.184772,12.307691,41.513401,11.198285,8.938420,11.615466,9.092172,24.692553,18.190470,34.389965,8.728005,58.154320,13.610037,9.82,32.367973,26.595936,28.854656,21

In [9]:
df.index = pd.to_datetime(df.index)

# Adquirindo os tickers

In [10]:
assets = df.columns

In [11]:
!pip install PyPortfolioOpt

     |████████████████████████████████| 51kB 4.9MB/s 


# Otimizando o portfólio

## Importando pacotes

In [12]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

## 1) Calcular retornos e matriz de var/cov anualizados a partir de dados diários

In [13]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

/usr/local/lib/python3.6/dist-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."


## 2) Otimizar o Índice de Sharpe Máximo (DE TODA A BOLSA)
##    Sharpe > 1 = aceitavel; nos parenteses embaixo sai o valor decimal das porcentagens apresentadas

In [14]:
ef = EfficientFrontier(mu, S) # cria a fronteira eficiente
weights = ef.max_sharpe() # maximiza os pesos ingênuos

cleaned_weights = ef.clean_weights() # considera alguns pesos como zero
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('ABEV3.SA', 0.0), ('AZUL4.SA', 0.0), ('B3SA3.SA', 0.0), ('BBAS3.SA', 0.0), ('BBDC3.SA', 0.0), ('BBDC4.SA', 0.0), ('BBSE3.SA', 0.0), ('BEEF3.SA', 0.0), ('BPAC11.SA', 0.02099), ('BRAP4.SA', 0.0), ('BRDT3.SA', 0.0), ('BRFS3.SA', 0.0), ('BRKM5.SA', 0.0), ('BRML3.SA', 0.0), ('BTOW3.SA', 0.0), ('CCRO3.SA', 0.0), ('CIEL3.SA', 0.0), ('CMIG4.SA', 0.0), ('COGN3.SA', 0.0), ('CPFE3.SA', 0.0), ('CRFB3.SA', 0.0), ('CSAN3.SA', 0.0), ('CSNA3.SA', 0.15265), ('CVCB3.SA', 0.0), ('CYRE3.SA', 0.0), ('ECOR3.SA', 0.0), ('EGIE3.SA', 0.0), ('ELET3.SA', 0.0), ('ELET6.SA', 0.0), ('EMBR3.SA', 0.0), ('ENBR3.SA', 0.0), ('ENGI11.SA', 0.0), ('EQTL3.SA', 0.0), ('FLRY3.SA', 0.0), ('GGBR4.SA', 0.0), ('GNDI3.SA', 0.0), ('GOAU4.SA', 0.0), ('GOLL4.SA', 0.0), ('HAPV3.SA', 0.0), ('HGTX3.SA', 0.0), ('HYPE3.SA', 0.0), ('IGTA3.SA', 0.0), ('IRBR3.SA', 0.0), ('ITSA4.SA', 0.0), ('ITUB4.SA', 0.0), ('JBSS3.SA', 0.0), ('KLBN11.SA', 0.0), ('LAME4.SA', 0.0), ('LREN3.SA', 0.0), ('MGLU3.SA', 0.17438), ('MRFG3.SA', 0.0), ('M

(1.1272687446558187, 0.4121718461798496, 2.686424982488169)

## 2.1) Hierarquizando os pesos por ação

In [15]:
!pip install pulp

     |████████████████████████████████| 40.6MB 106kB/s 


In [16]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices # Alocações discreta

portfolio_val = 4000 # quantos dolares eu quero colocar no portfolio
latest_prices = get_latest_prices(df)
weights = cleaned_weights # pesos estimados acima
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value= portfolio_val) # discrete allocation
allocation, leftover = da.lp_portfolio() # armazena os dados redigidos em duas listas, uma com as alocações e outra com o que sobrou do dinheiro
print('DiscreteAllocation:', allocation)
print('Funds Remaing: U$', leftover)

DiscreteAllocation: {'BPAC11.SA': 1, 'BRML3.SA': 1, 'CSNA3.SA': 17, 'MGLU3.SA': 27, 'RADL3.SA': 1, 'WEGE3.SA': 33}
Funds Remaing: U$ 0.033461570739746094


# 3) Fornecendo os resultados comnome das comp.

In [17]:
def get_company_name(symbol): # função que fornece o nome delas
  url = 'http://d.yimg.com/autoc.finance.yahoo.com/autoc?query='+symbol+'&region=1&lang=en'
  result = requests.get(url).json()
  for r in result['ResultSet']['Result']:
    if r ['symbol']==symbol:
      return r['name']

In [18]:
company_name=[] # retornando os nomes em uma lista
for symbol in allocation:
  company_name.append( get_company_name(symbol) )

In [19]:
discrete_allocation_list = [] # adquirindo o valor de alocação discreta
for symbol in allocation:
  discrete_allocation_list.append(allocation.get(symbol))

In [20]:
portfolio_df = pd.DataFrame(columns= ['Company_Name', 'Company_Ticker', 'Discrete_Val'+str(portfolio_val)]) # Criando um DataFrame para o portfólio

In [21]:
portfolio_df['Company_Name'] = company_name
portfolio_df['Company_Ticker'] = allocation
portfolio_df['Discrete_val'+str(portfolio_val)] = discrete_allocation_list

In [22]:
portfolio_df = portfolio_df.dropna(axis=1)

In [23]:
portfolio_df# mostrando o porfolio

,Company_Name,Company_Ticker,Discrete_val4000
0,Banco BTG Pactual S.A.,BPAC11.SA,1
1,BR Malls Participações S.A.,BRML3.SA,1
2,Companhia Siderúrgica Nacional,CSNA3.SA,17
3,Magazine Luiza S.A.,MGLU3.SA,27
4,Raia Drogasil S.A.,RADL3.SA,1
5,WEG S.A.,WEGE3.SA,33
